In [ ]:
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import zscore

In [ ]:
base_folder = Path('./result/readability-indexes')

scaler = MinMaxScaler()

generated_simplified = pd.read_csv(base_folder / 'generated-simplified.csv')
reference_simplified  = pd.read_csv(base_folder / 'reference-simplified.csv')
reference_complete = pd.read_csv(base_folder / 'reference-complete.csv')

In [ ]:
reference = pd.concat([reference_complete,reference_simplified])
reference = reference[reference["model"]!= "pt_core_news_lg"]

generated_simplified = generated_simplified[generated_simplified["model"]!= "pt_core_news_lg"]

In [ ]:
merged = reference.merge(generated_simplified,on='name',how='inner',suffixes=("",'_generated'))

In [ ]:
scale_readability_cols = ['flesch_ease','gulpease'] 
grade_level_readability_cols = ['flesch_kincaid','ari','gunning_fog','coleman_liau']
metrics_cols = scale_readability_cols + grade_level_readability_cols


### Teste Chi-Quadrado

### Teste T

In [ ]:
import numpy as np 
import scipy.stats as st
import csv
from typing import Any
import math

confidence = 0.95
confidence_intervals: list[dict[str,Any]] = []

for metric in grade_level_readability_cols:
    per_group = merged.groupby("generated_with_generated")[f"{metric}_generated"]
    for row in per_group.agg(["mean","count",st.sem]).iterrows():
        range_min,range_max = st.t.interval(confidence=confidence,df=row[1]["count"]-1,loc=row[1]["mean"],scale=row[1]["sem"])
        confidence_intervals.append({"metric":metric,"model":row[0], "range_min":range_min,"mean": row[1]["mean"], "range_max":range_max})

    original = merged[metric].unique()
    mean = original.mean()
    sem = st.sem(original)
    range_min,range_max = st.t.interval(confidence=confidence,df=len(original)-1,loc=mean,scale=sem)
    confidence_intervals.append({"metric":metric,"model":"Original", "range_min":range_min,"mean": mean, "range_max":range_max})


    for model in merged["generated_with_generated"].unique():
        print(st.ttest_ind(original, per_group.get_group(model),alternative="greater"))

df_grade_level = pd.DataFrame(confidence_intervals)
df_grade_level[["range_min","mean","range_max"]] = df_grade_level[["range_min","mean","range_max"]].map(lambda x: math.trunc(100*x) / 100)
df_grade_level.to_csv("confidence_intervals_indexes_grade_level.csv",index=False)

confidence_intervals = []
for metric in scale_readability_cols:
    per_group = merged.groupby("generated_with_generated")[f"{metric}_generated"]
    for row in per_group.agg(["mean","count",st.sem]).iterrows():
        range_min,range_max = st.t.interval(confidence=confidence,df=row[1]["count"]-1,loc=row[1]["mean"],scale=row[1]["sem"])
        confidence_intervals.append({"metric":metric,"model":row[0], "range_min":range_min,"mean": row[1]["mean"], "range_max":range_max})

    original = merged[metric].unique()
    mean = original.mean()
    sem = st.sem(original)
    range_min,range_max = st.t.interval(confidence=confidence,df=len(original)-1,loc=mean,scale=sem)
    confidence_intervals.append({"metric":metric,"model":"Original", "range_min":range_min,"mean": mean, "range_max":range_max})

df_hundred_scale = pd.DataFrame(confidence_intervals)
df_hundred_scale[["range_min","mean","range_max"]] = df_hundred_scale[["range_min","mean","range_max"]].map(lambda x: math.trunc(100*x) / 100)
df_hundred_scale.to_csv("confidence_intervals_indexes_hundred_scale.csv",index=False)


In [ ]:
# generate a box plot with the confidence intervals, add a red line for the mean
# using the confidence_intervals dictionary
# each metric should be a separate chart
import matplotlib.pyplot as plt
import seaborn as sns


for metric in grade_level_readability_cols:
    plt.figure(figsize=(14,6))
    sns.boxplot(x="generated_with_generated", y=f"{metric}_generated", data=merged)
    
    for group in merged["generated_with_generated"].unique()+["original"]:
        if (metric,group) in confidence_intervals:
            range_min, mean, range_max = confidence_intervals[metric,group]
            plt.plot([group], [mean], marker='o', markersize=10, color='red')
            plt.vlines(x=group, ymin=range_min, ymax=range_max, color='red', linestyle='--')
    
    plt.title(f'Box plot of {metric} with Confidence Intervals')
    plt.ylabel(metric)
    plt.xlabel('Generated With Generated')
    plt.grid(True)
    plt.xticks(rotation=45, ha='right')
    plt.xticks(fontsize=10)
    plt.show()